## JHU COVID-19 Data Pull and Clean Script

In [1]:
# Bring in necessary libraries
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

import sys, os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

os.getcwd()

'C:\\Users\\Neil\\Documents\\Projects\\COVID-19\\COVID-19\\scripts'

In [2]:
# Process time-series data
c_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_19-covid-Confirmed_archived_0325.csv'
c_g_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
d_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_19-covid-Deaths_archived_0325.csv'
d_g_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
r_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_19-covid-Recovered_archived_0325.csv'

c = pd.read_csv(c_url)
c_g = pd.read_csv(c_g_url)
d = pd.read_csv(d_url)
d_g = pd.read_csv(d_g_url)
r = pd.read_csv(r_url)

c_g.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,174,237,273,281,299,349,367,423,444,484
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,243,259,277,304,333,361,377,383,400,409
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,716,847,986,1171,1251,1320,1423,1468,1572,1666
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,376,390,428,439,466,501,525,545,564,583
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,7,8,8,8,10,14,16,17,19,19


In [3]:
# Unpivot dataframes
c = c.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'date', value_name = 'confirmed')
c_g = c_g.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'date', value_name = 'confirmed')
d = d.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'date', value_name = 'deaths')
d_g = d_g.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'date', value_name = 'deaths')
r = r.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'date', value_name = 'recovered')

# Convert from string to date
c['date'] = pd.to_datetime(c['date'])
c_g['date'] = pd.to_datetime(c_g['date'])
d['date'] = pd.to_datetime(d['date'])
d_g['date'] = pd.to_datetime(d_g['date'])
r['date'] = pd.to_datetime(r['date'])

# Remove US from c_g and d_g
c_g = c_g[c_g['Country/Region']!='US']
d_g = d_g[d_g['Country/Region']!='US']

# Only keep the 50 states and a few special regions (get rid of the county level data) from c, g, and r
states_list = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 
               'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 
               'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 
               'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
               'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 
               'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 
               'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 
               'Wyoming', 'American Samoa', 'Diamond Princess', 'Grand Princess', 'Guam', 
               'Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands']

c = c[(c['Country/Region']!='US') | ((c['Country/Region']=='US') & (c['Province/State'].isin(states_list)))]
d = d[(d['Country/Region']!='US') | ((d['Country/Region']=='US') & (d['Province/State'].isin(states_list)))]
r = r[(r['Country/Region']!='US') | ((r['Country/Region']=='US') & (r['Province/State'].isin(states_list)))]

# Truncate c,d, and r to end in 3/22 - keep only US in c and d
c = c[(c['date'] <= '2020-03-22') & (c['Country/Region']=='US')]
d = d[(d['date'] <= '2020-03-22') & (d['Country/Region']=='US')]
r = r[r['date'] <= '2020-03-22']

# Get state locations from c
#states_loc = c.groupby('Province/State', as_index=False).agg({'Lat': 'mean', 'Long': 'mean'})
#states_list = ['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Minor Outlying Islands', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'U.S. Virgin Islands', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
#states_loc = states_loc[states_loc['Province/State'].isin(states_list)]

# Merge and append everything together
df = c_g.merge(d_g, how='outer', left_on = ['Province/State', 'Country/Region', 'Lat', 'Long', 'date'],
                    right_on = ['Province/State', 'Country/Region', 'Lat', 'Long', 'date'])
df = df.merge(r, how='outer', left_on = ['Province/State', 'Country/Region', 'Lat', 'Long', 'date'],
                    right_on = ['Province/State', 'Country/Region', 'Lat', 'Long', 'date'])
df = df.append(c)
df = df.append(d)

# Get the latitudes and longitudes of every province/state in the dataframe
locations = df.groupby(['Country/Region', 'Province/State'], as_index=False).agg({'Lat': 'mean', 'Long': 'mean'})

df.head()

C:\Users\Neil\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Country/Region,Lat,Long,Province/State,confirmed,date,deaths,recovered
0,Afghanistan,33.0000,65.0000,NaN,0.0,2020-01-22,0.0,0.0
1,Albania,41.1533,20.1683,NaN,0.0,2020-01-22,0.0,0.0
2,Algeria,28.0339,1.6596,NaN,0.0,2020-01-22,0.0,0.0
3,Andorra,42.5063,1.5218,NaN,0.0,2020-01-22,0.0,0.0
4,Angola,-11.2027,17.8739,NaN,0.0,2020-01-22,0.0,0.0


In [4]:
# Get the most recent daily reports data
# First see if there is a file for today

try:
    del daily
except:
    print("")

daily_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+date.today().strftime("%m-%d-%Y")+'.csv'
try:
    daily = pd.read_csv(daily_url)
    daily['date'] = date.today()
except:
    print('No daily file available yet for today')
day = date.today() - timedelta(days=1)

# If no daily report file for today, try yesterday's
try:
    daily
except:   
    daily_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+(date.today() - timedelta(days=1)).strftime("%m-%d-%Y")+'.csv'
    daily = pd.read_csv(daily_url)
    daily['date'] = date.today() - timedelta(days=1)
    day = date.today() - timedelta(days=2)
    
#print(type(day))
#day > datetime(2020, 3, 22).date()

# Loop through days and get the remaining data
while day > datetime(2020, 3, 22).date():
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+day.strftime("%m-%d-%Y")+'.csv'
    temp = pd.read_csv(url)
    temp['date'] = day
    daily = daily.append(temp)
    day = day - timedelta(days=1)
    
# Convert Last_Update to date
# Drop unnecessary columns
daily = daily.drop(columns=['Last_Update', 'Combined_Key', 'Active'])
daily = daily.rename(columns={'Long_': 'Long', 'Admin2': 'County', 'Province_State': 'Province/State', 
                              'Country_Region': 'Country/Region', 'Recovered': 'recovered', 
                              'Confirmed': 'confirmed', 'Deaths': 'deaths'})

# Remove the US and Canada total recovered rows
daily = daily[daily['Province/State']!='Recovered']

# Fix Falkland Islands name change
daily['Province/State'][daily['Province/State']=='Falkland Islands (Islas Malvinas)'] = 'Falkland Islands (Malvinas)'

daily.head()


No daily file available yet for today


,FIPS,County,Province/State,Country/Region,Lat,Long,confirmed,deaths,recovered,date
0,45001.0,Abbeville,South Carolina,US,34.223334,-82.461707,7,0,0,2020-04-09
1,22001.0,Acadia,Louisiana,US,30.295065,-92.414197,89,3,0,2020-04-09
2,51001.0,Accomack,Virginia,US,37.767072,-75.632346,11,0,0,2020-04-09
3,16001.0,Ada,Idaho,US,43.452658,-116.241552,447,5,0,2020-04-09
4,19001.0,Adair,Iowa,US,41.330756,-94.471059,1,0,0,2020-04-09


In [5]:
# Maniuplate the daily report

# Create a US by county dataset
us_counties = daily.loc[(daily['Country/Region']=="US") & (daily['County'].notna())]
us_counties['active'] = us_counties['confirmed'] - us_counties['deaths'] - us_counties['recovered']

# Remove US counties from daily dataset
daily_us = daily.loc[(daily['Country/Region']=='US') & (daily['County'].isna())]
daily_us = daily_us.drop(columns=['County', 'FIPS'])
daily = daily.loc[daily['Country/Region']!='US']
daily = daily.drop(columns=['FIPS'])
daily = daily.rename(columns={'recovered': 'recovered_new'})


C:\Users\Neil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [6]:
# Aggregate US county data to state level
states = us_counties.groupby(['Province/State', 'date'], as_index=False).agg({'confirmed': 'sum', 'deaths': 'sum', 'recovered': 'sum'})
#states['Country/Region'] = "US"

# Bring in latitudes and longitudes from States_loc
states = states.merge(locations, how='inner', on=['Province/State'])
states.head()


,Province/State,date,confirmed,deaths,recovered,Country/Region,Lat,Long
0,Alabama,2020-03-23,196,0,0,US,32.3182,-86.9023
1,Alabama,2020-03-24,242,0,0,US,32.3182,-86.9023
2,Alabama,2020-03-25,381,1,0,US,32.3182,-86.9023
3,Alabama,2020-03-26,517,1,0,US,32.3182,-86.9023
4,Alabama,2020-03-27,587,4,0,US,32.3182,-86.9023


In [7]:
# Merge in recovered for non-US regions
daily = daily.drop(columns=['County', 'confirmed', 'deaths', 'Lat', 'Long'])
df['date'] = pd.to_datetime(df['date']).dt.date
df = df.merge(daily, how='outer', on=['Country/Region', 'Province/State', 'date'])

# Replace NA's in recovered column with new recovered values
df['recovered'].fillna(df['recovered_new'], inplace=True)
df = df.drop(columns=['recovered_new'])

In [8]:
# Merge in lat's and long's from locations df to the daily_us df
#daily_us = daily_us.merge(locations, how='inner', on=['Province/State'])

# Append daily_us and states to the df
df = df.append(daily_us)
df = df.append(states)

In [9]:
# Replace NAs with 0s, convert from floats to ints

df['deaths'].fillna(0, inplace=True)
df['deaths'] = df['deaths'].astype(int)
df['confirmed'].fillna(0, inplace=True)
df['confirmed'] = df['confirmed'].astype(int)
df['recovered'].fillna(0, inplace=True)
df['recovered'] = df['recovered'].astype(int)

# Correct the lat and long for the cruise ships in the newer data
df.loc[(df['Country/Region']=='Cruise Ship') & (df['Province/State']=='Diamond Princess'), 'Lat'] = df[(df['Province/State']=='Diamond Princess') & (df['Country/Region']=='Cruise Ship') & (df['date']<datetime(2020,3,23).date())].groupby(['Country/Region', 'Province/State']).agg({'Lat': 'mean'})['Lat'][0]
df.loc[(df['Country/Region']=='Cruise Ship') & (df['Province/State']=='Diamond Princess'), 'Long'] = df[(df['Province/State']=='Diamond Princess') & (df['Country/Region']=='Cruise Ship') & (df['date']<datetime(2020,3,23).date())].groupby(['Country/Region', 'Province/State']).agg({'Long': 'mean'})['Long'][0]
df.loc[(df['Country/Region']=='US') & (df['Province/State']=='Diamond Princess'), 'Lat'] = df[(df['Province/State']=='Diamond Princess') & (df['Country/Region']=='US') & (df['date']<datetime(2020,3,23).date())].groupby(['Country/Region', 'Province/State']).agg({'Lat': 'mean'})['Lat'][0]
df.loc[(df['Country/Region']=='US') & (df['Province/State']=='Diamond Princess'), 'Long'] = df[(df['Province/State']=='Diamond Princess') & (df['Country/Region']=='US') & (df['date']<datetime(2020,3,23).date())].groupby(['Country/Region', 'Province/State']).agg({'Long': 'mean'})['Long'][0]
df.loc[(df['Country/Region']=='US') & (df['Province/State']=='Grand Princess'), 'Lat'] = df[(df['Province/State']=='Grand Princess') & (df['Country/Region']=='US') & (df['date']<datetime(2020,3,23).date())].groupby(['Country/Region', 'Province/State']).agg({'Lat': 'mean'})['Lat'][0]
df.loc[(df['Country/Region']=='US') & (df['Province/State']=='Grand Princess'), 'Long'] = df[(df['Province/State']=='Grand Princess') & (df['Country/Region']=='US') & (df['date']<datetime(2020,3,23).date())].groupby(['Country/Region', 'Province/State']).agg({'Long': 'mean'})['Long'][0]

#df.loc[(df['Country/Region']=='United Kingdom') & (df['Province/State']=='Grand Princess'), 'Lat'] = df[(df['Province/State']=='Grand Princess') & (df['Country/Region']=='US') & (df['date']<datetime(2020,3,23).date())].groupby(['Country/Region', 'Province/State']).agg({'Lat': 'mean'})['Lat'][0]
#df.loc[(df['Country/Region']=='United Kingdom') & (df['Province/State']=='Grand Princess'), 'Long'] = df[(df['Province/State']=='Grand Princess') & (df['Country/Region']=='US') & (df['date']<datetime(2020,3,23).date())].groupby(['Country/Region', 'Province/State']).agg({'Long': 'mean'})['Long'][0]



df = df.round({'Lat': 3, 'Long': 3})

df = df.sort_values(by=['Country/Region', 'Province/State', 'date'])

In [10]:
# Limit US counties to most recent day
us_counties = us_counties[us_counties['date']==us_counties['date'].max()]
us_counties = us_counties.sort_values(by=['County'])

# Convert FIPS to strings
us_counties = us_counties[us_counties['FIPS'].notna()]
us_counties['FIPS'] = us_counties['FIPS'].astype(int).astype(str)

In [11]:
# Export csv and remove SQL results
os.chdir('..')
df.to_csv('ts.csv', index=False)
us_counties.to_csv('counties.csv', index=False)

try:
    os.remove('.\\output\\region_latest.csv')
except:
    print("File doesn't exist")
try:
    os.remove('.\\output\\region_province_ts.csv')
except:
    print("File doesn't exist")
try:
    os.remove('.\\output\\ts_chart.csv')
except:
    print("File doesn't exist")

os.chdir(".\\scripts")